### Import necessary modules

In [1]:
import json
import pathlib
import requests
import time
# twittertools is my local twittertools.py module
import twittertools

### Define Elasticsearch server URI

In [2]:
elasticsearch_server = 'http://localhost:9200'

In [3]:
# Print current indices
r = requests.get(elasticsearch_server + '/_cat/indices')
print(r.text)

yellow open twitter UF9VLRm_SHOoExEiWG0pUg 5 1 3242 0  2.4mb  2.4mb
yellow open .kibana Yuy-vovlQyeUPVim2PcdFw 1 1    2 0 10.9kb 10.9kb



### Create Authenticated TwitterTools object

In [4]:
filepath = pathlib.Path.home().joinpath('.twitter', 'credentials.json')
twt = twittertools.TwitterTools(filepath)

### Get tweets from @pourmecoffee's timeline

In [5]:
screen_name = 'pourmecoffee'
tweets = twt.get_user_timeline(screen_name)
print(f"{len(tweets)} tweets retrieved from @{screen_name}'s timeline")

3242 tweets retrieved from @pourmecoffee's timeline


### Add all tweets to /twitter/tweets

In [6]:
# For this demonstration only, delete any existing /twitter index
tweet_index_uri = elasticsearch_server + '/twitter'
requests.delete(tweet_index_uri)

# Index the tweets
post_uri = tweet_index_uri + '/tweets'
success = 0
for tweet in tweets:
    source = twittertools.unpack_tweet(tweet)
    r = requests.post(post_uri, json=source)
    if r.status_code in [200, 201]:
        success += 1
print(f'{success} tweets indexed')

3242 tweets indexed


### Confirm number of tweets added to /twitter/tweets

In [7]:
sleep_time = 0.250
search_uri = post_uri + '/_search?q=*'
while True:
    r = requests.get(search_uri)
    results = dict(r.json())
    total = results['hits']['total']
    if total != success:
        print(f'Found {total} of {success} tweets. Sleeping for {sleep_time} seconds.')
        time.sleep(sleep_time)
    else:
        print(f'Found all {total} tweets!')
        break

Found 3131 of 3242 tweets. Sleeping for 0.25 seconds.
Found 3131 of 3242 tweets. Sleeping for 0.25 seconds.
Found all 3242 tweets!
